In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

class HateSpeechDetector:
    def __init__(self, model_path="./model"):
        """
        Initialize the hate speech detector.

        Args:
            model_path (str): Path to the model directory (the folder containing all model files)
        """
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_path)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model.eval()

        self.id2label = self.model.config.id2label
        print(f"Model loaded successfully!")
        print(f"Classes: {list(self.id2label.values())}")
        print(f"Device: {self.device}")

    def predict(self, text):
        """
        Predict the class of the text.

        Args:
            text (str): Input text to classify

        Returns:
            dict: Prediction results with confidence scores
        """
        text = str(text).strip()
        if not text:
            return {"error": "Empty text provided"}

        inputs = self.tokenizer(
            text,
            truncation=True,
            padding=True,
            max_length=128,
            return_tensors='pt'
        )

        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = self.model(**inputs)

        probabilities = torch.softmax(outputs.logits, dim=-1)
        prediction_idx = torch.argmax(probabilities, dim=-1).item()
        confidence = probabilities[0][prediction_idx].item()

        prob_dict = {}
        for idx, prob in enumerate(probabilities[0]):
            label_name = self.id2label[idx]
            prob_dict[label_name] = round(prob.item(), 4)

        return {
            'text': text,
            'prediction': self.id2label[prediction_idx],
            'confidence': round(confidence, 4),
            'probabilities': prob_dict
        }

    def predict_batch(self, texts):
        """Predict multiple texts at once"""
        return [self.predict(text) for text in texts]

    def analyze_texts(self, texts):
        """Analyze multiple texts with formatted output"""
        print("Analyzing Texts...")
        print("=" * 60)

        for i, text in enumerate(texts, 1):
            result = self.predict(text)
            print(f"\n{i}. Text: {text}")
            print(f"   Prediction: {result['prediction'].upper()}")

if __name__ == "__main__":
    detector = HateSpeechDetector(model_path="./model")

    test_texts = [
        "I love spending time with my family and friends",
        "Women are useless and belong in the kitchen not in workplaces",
        "prince is bitch",
        "you faggot son of a bitch, you should die by now",
        "I hate all immigrants they should leave our country immediately",
        "The weather is beautiful today, let's go for a walk",
        "Get rid of all the Jews from our society they are parasites",
        "a negro honey trap",
        "What a moronic thing to say, are you brain dead?",
        "I'm really frustrated with this computer not working properly",
        "Aakash may go to hell, lets your small ass get bit by that fucking girlfired of yours",
        "you nigger fuck, go to hell",
        "you say beautiful"
        
    ]

    detector.analyze_texts(test_texts)